In [ ]:
import numpy as np
from scipy.fft import fft, ifft, fftshift
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import pickle
from copy import copy
%matplotlib inline

In [ ]:
with open('data_set_1.pkl', 'rb') as f:
    data_set_1 = pickle.load(f)

xvals1 = data_set_1[:, 0]
yvals1 = data_set_1[:, 1]

In [ ]:
def xp_mat_maker(xvals, p):
    xp = np.ones((xvals.size, p+1))
    for jj in range(p):
        xp[:, jj+1] = xvals * xp[:, jj]
    return xp

In [ ]:
def least_squares_solve(mat, yvals):
    u, s, vt = np.linalg.svd(mat, full_matrices=False)
    alpha = (vt.T @ np.diag(1./s) @ u.T) @ yvals.reshape(-1, 1)
    error = np.linalg.norm(mat @ alpha - yvals.reshape(-1, 1))**2./(2.*yvals.size)
    return alpha, error 

In [ ]:
def cross_fold(alpha, xvals, yvals, pval, kfolds):
    inds = np.arange(yvals.size)
    alpha_avg = np.zeros(pval+1)
    np.random.shuffle(inds)
    train_range = yvals[inds]
    train_domain = xvals[inds]
    bucket_size = int(np.floor(yvals.size/kfolds))
    for jj in range(kfolds):
        bvec_train = train_range[jj*bucket_size:(jj+1)*bucket_size]
        xp_train = xp_mat_maker(train_domain[jj*bucket_size:(jj+1)*bucket_size], pval)        
        res = Lasso(alpha, max_iter=4000).fit(xp_train, bvec_train) # alpha > 0
        alpha_lasso = res.coef_
        alpha_avg += alpha_lasso
    alpha_avg /= kfolds
    return alpha_avg
    
def error_testing(alpha, xvals, yvals, pval):    
    num_experiments = 1000    
    errors = np.zeros(num_experiments)        
    inds = np.arange(yvals.size)
    expsize = round(.1*yvals.size)
    for jj in range(num_experiments):
        np.random.shuffle(inds)
        shuffle_range = yvals[inds]
        bvec = shuffle_range[:expsize].reshape(-1,1)
        shuffle_domain = xvals[inds]
        xp = xp_mat_maker(shuffle_domain[:expsize], pval)
        errors[jj] = np.linalg.norm(xp@alpha.reshape(-1, 1) - bvec)     
    mean_err = np.mean(errors)
    stand_dev = np.std(errors)
    return mean_err, stand_dev

In [ ]:
plt.scatter(xvals1, yvals1, s=1.)

In [ ]:
# rescale data
xmax = np.max(xvals1)
ymax = np.max(yvals1)
ymin = np.min(yvals1)
xvalsrsc = xvals1/xmax
yvalsrsc = (yvals1-ymin)/(ymax-ymin)
plt.scatter(xvalsrsc, yvalsrsc, s=1.)

In [ ]:
pval = 4
xp = xp_mat_maker(xvalsrsc, pval)
alpha, error = least_squares_solve(xp, yvalsrsc)
yfit = xp @ alpha.reshape(-1,1)
plt.plot(xvalsrsc, yfit, color='r')
plt.scatter(xvalsrsc, yvalsrsc, s=1.)

In [ ]:
yfit = (ymax-ymin)*yfit + ymin
plt.plot(xvals1,yfit,color='r')
plt.scatter(xvals1, yvals1, s=1.)

In [ ]:
numlams = 30
lampows = np.linspace(-4., -1.5, numlams) # choices, choices
errors = np.zeros(lampows.size)
stds = np.zeros(lampows.size)
pval = 8
kfolds = 10
train_size = round(.9*yvalsrsc.size)
avalsvslam = np.zeros((pval+1, numlams))
for jj, lampow in enumerate(lampows):
    lam = 10**(lampow)
    xtrain = xvalsrsc[:train_size]
    ytrain = yvalsrsc[:train_size]
    xtest = xvalsrsc[train_size:]
    ytest = yvalsrsc[train_size:]
    
    alpha_avg = cross_fold(lam, xtrain, ytrain, pval, kfolds)
    mean, std = error_testing(alpha_avg, xtest, ytest, pval) # alpha = 10^pow
    avalsvslam[:, jj] = alpha_avg
    errors[jj] = mean
    stds[jj] = std

In [ ]:
plt.plot(lampows, errors) # how would you plot the standard devations as error bars around the mean of the error?
for jj in range(stds.size):
    ejp = errors[jj] + stds[jj]
    ejm = errors[jj] - stds[jj]
    plt.plot([lampows[jj], lampows[jj]], [ejm, ejp], ls='--', c='r')
plt.xlabel(r"$\log_{10}(\lambda)$");
plt.ylabel(r"Mean Error")

In [ ]:
cfig = plt.pcolormesh(lampows, np.arange(pval+1), avalsvslam)
plt.xlabel(r"$\log_{10}(\lambda)$")
plt.ylabel(r"$\alpha_{j}$")
plt.colorbar(cfig);